In [1]:
import numpy as np
import pandas as pd
import argparse
import os

import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F

from custom_dataset import CustomDataset
from network import Network
from utils import *

from sklearn.model_selection import StratifiedShuffleSplit

In [2]:
torch.cuda.device_count()

1

In [3]:
parent_directory = '/data/users2/pnadigapusuresh1/JobOutputs'
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cuda device


### Loading model for 1 Fully-connected layer. 

In [4]:

model = Network()
#model = nn.DataParallel(model)
model.to(device)

# Loading the model from Job 5436878
#loading model from 6066159
load_path = os.path.join(parent_directory,'5436878','models','epoch_28')

model.load_state_dict(torch.load(load_path))

<All keys matched successfully>

In [5]:
model = nn.DataParallel(model)

In [6]:
torch.manual_seed(52)
np.random.seed(52)
# number of subprocesses to use for data loading
num_workers = 1
# how many samples per batch to load
batch_size = 5

In [7]:
valid_size = 0.20
# percentage of data to be used for testset
test_size = 0.10


train_data = CustomDataset(transform = 
                        transforms.Compose([
                            transforms.RandomHorizontalFlip()
                            ]),train=True)

valid_data = CustomDataset(train=False)

# get filtered variables
vars = valid_data.vars

In [8]:
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
test_split = int(np.floor(test_size * num_train))
test_idx, train_idx = indices[: test_split], indices[test_split : ]

train_rem = len(train_idx)
valid_spilt = int(np.floor(valid_size * train_rem))

valid_idx, train_idx = indices[: valid_spilt], indices[valid_spilt : ]

In [9]:
test_sampler = SubsetRandomSampler(test_idx)
test_loader = DataLoader(valid_data,batch_size = batch_size, 
                            sampler = test_sampler, num_workers=num_workers)

In [10]:
criterion = nn.L1Loss()

In [11]:
def test(model):
    actual_test = torch.tensor([]).to(device)
    pred_test = torch.tensor([]).to(device)

    test_loss = 0

    for X, y in test_loader:
        X,y = X.to(device),y.to(device)
        actual_test = torch.cat((actual_test,y),0)
        pred = torch.squeeze(model(torch.unsqueeze(X,1).float()))
        try:
            test_loss += criterion(pred,y.float()).item()
        except:
            print(pred)
            print(y)
    
    return test_loss/len(test_loader)

### 1 Fc Layer/Folds

### Testing for 1 FC layers

In [30]:
train_scores = [3.014,2.927,2.990,2.935,2.883]
val_scores = [2.960,3.033,3.071,2.920,3.011]

print('train mean =',np.mean(train_scores))
print('train std =',np.std(train_scores))
print('val mean =',np.mean(val_scores))
print('val std =',np.std(val_scores))

train mean = 2.9498000000000006
train std = 0.04677349676900367
val mean = 2.999
val std = 0.0533778980477876


In [12]:
model = Network()
model.fc1 = nn.Sequential(nn.Linear(512,1))
model = nn.DataParallel(model)
model.to(device)

<All keys matched successfully>

In [14]:
best_models_1fc = {
    '1' : 'epoch_65',
    '2' : 'epoch_71',
    '3' : 'epoch_65',
    '4' : 'epoch_71',
    '5' : 'epoch_75'
}

In [19]:
test_scores = []
for k,v in best_models_1fc.items():
    load_path = os.path.join(parent_directory,'6164265/models_fold',k,v)
    model.load_state_dict(torch.load(load_path))
    test_scores.append(test(model))

print('mean = ',np.mean(test_scores))
print('std = ',np.std(test_scores)) 

mean =  3.274067613364391
std =  0.21072294520415621


### Testing for 2 Fc layers

In [31]:
train_scores = [4.956, 4.413, 4.980, 4.451, 4.669]
val_scores = [4.854, 4.495, 4.834, 4.397, 4.498]

print('train mean =',np.mean(train_scores))
print('train std =',np.std(train_scores))
print('val mean =',np.mean(val_scores))
print('val std =',np.std(val_scores))

train mean = 4.6938
train std = 0.24044741628888447
val mean = 4.6156
val std = 0.1901016570153978


In [22]:
model = Network()
model.fc1 = nn.Sequential(nn.Linear(512,256),nn.ReLU(),
    nn.Dropout(),nn.Linear(256,1))
model = nn.DataParallel(model)
model.to(device)

DataParallel(
  (module): Network(
    (cv1): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv2): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv3): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn3): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv4): Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn4): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv5): Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn5): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pool): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (fc1): Sequential(
      (0): Linear(in_features=512, out_features=256, bias=

In [23]:
best_models_2fc = {
    '1' : 'epoch_26',
    '2' : 'epoch_74',
    '3' : 'epoch_25',
    '4' : 'epoch_66',
    '5' : 'epoch_48'
}

In [25]:
test_scores = []
for k,v in best_models_2fc.items():
    load_path = os.path.join(parent_directory,'6173079/models_fold',k,v)
    model.load_state_dict(torch.load(load_path))
    test_scores.append(test(model))

print('mean = ',np.mean(test_scores))
print('std = ',np.std(test_scores)) 

mean =  5.720739652510709
std =  0.8713915309932332


### Testing for 3 Fc layers

In [32]:
train_scores = [7.314, 6.267, 6.291, 6.107, 6.374]
val_scores = [8.838, 6.731, 6.394, 5.996, 6.227]

print('train mean =',np.mean(train_scores))
print('train std =',np.std(train_scores))
print('val mean =',np.mean(val_scores))
print('val std =',np.std(val_scores))

train mean = 6.4706
train std = 0.4304809403446335
val mean = 6.837200000000001
val std = 1.0286755367947655


In [26]:
model = Network()
model.fc1 = nn.Sequential(nn.Linear(512,256),nn.ReLU(),
    nn.Dropout(),nn.Linear(256,128),nn.ReLU(),nn.Dropout(),
    nn.Linear(128,1))
model = nn.DataParallel(model)
model.to(device)

DataParallel(
  (module): Network(
    (cv1): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv2): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv3): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn3): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv4): Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn4): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv5): Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn5): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pool): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (fc1): Sequential(
      (0): Linear(in_features=512, out_features=256, bias=

In [27]:
best_models_3fc = {
    '3' : 'epoch_62',
    '4' : 'epoch_75',
    '5' : 'epoch_51'
}

In [29]:
test_scores = []
for k,v in best_models_3fc.items():
    load_path = os.path.join(parent_directory,'6183834/models_fold',k,v)
    model.load_state_dict(torch.load(load_path))
    test_scores.append(test(model))

print('mean = ',np.mean(test_scores))
print('std = ',np.std(test_scores))

mean =  7.953095125702192
std =  0.3487180256342292
